In [10]:
import sys
import sqlite3
import pandas as pd
import re
from unicodedata import normalize
from PyPDF2 import PdfReader, PageObject
import io
import importlib

import yhtokenize

import MeCab
import ipadic

In [2]:
def db_con() -> sqlite3.Connection:
    path = "../ffgcrawl/db.sqlite"
    con: sqlite3.Connection = sqlite3.connect(path)
    con.row_factory = sqlite3.Row
    return con

In [3]:
def query(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, db_con())

In [4]:
pdfs = query("select * from pdfs limit 10")

In [5]:
pdfs.head()

,id,url,referrer_title,content,created_at
0,1,https://www.fukuoka-fg.com/news_info_pdf/2023/...,ふくおかフィナンシャルグループ,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,2023-04-05T14:25:21.296255
1,2,https://www.fukuoka-fg.com/news_info_pdf/2023/...,ふくおかフィナンシャルグループ,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,2023-04-05T14:25:21.378758
2,3,https://www.fukuoka-fg.com/news_info_pdf/2023/...,ふくおかフィナンシャルグループ,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,2023-04-05T14:25:21.451051
3,4,https://www.fukuoka-fg.com/news_info_pdf/2023/...,ふくおかフィナンシャルグループ,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,2023-04-05T14:25:21.642081
4,5,https://www.fukuoka-fg.com/news_info_pdf/2023/...,ふくおかフィナンシャルグループ,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,2023-04-05T14:25:21.698366


In [6]:
importlib.reload(yhtokenize)

<module 'yhtokenize' from '/Users/hotoku/projects/hotoku/ffg-crawl/works/yhtokenize.py'>

In [14]:
_TAGGER = MeCab.Tagger(ipadic.MECAB_ARGS)

In [33]:
ret = _TAGGER.parse(yhtokenize.extract(pdfs.content.iloc[0]))
words = ret.split("\n")
print("\n".join((map(lambda ss: "\t".join(ss), map(lambda s: re.split(r"[\t,]", s), words)))))

NewsRelease	名詞	固有名詞	組織	*	*	*	*
〒	記号	一般	*	*	*	*	〒	ユウビンバンゴウ	ユービンバンゴー
810	名詞	数	*	*	*	*	*
-	記号	一般	*	*	*	*	*
8693	名詞	数	*	*	*	*	*
福岡	名詞	固有名詞	地域	一般	*	*	福岡	フクオカ	フクオカ
市	名詞	接尾	地域	*	*	*	市	シ	シ
中央	名詞	固有名詞	地域	一般	*	*	中央	チュウオウ	チューオー
区	名詞	接尾	地域	*	*	*	区	ク	ク
大手門	名詞	固有名詞	地域	一般	*	*	大手門	オオテモン	オーテモン
1	名詞	数	*	*	*	*	*
-	記号	一般	*	*	*	*	*
8	名詞	数	*	*	*	*	*
-	記号	一般	*	*	*	*	*
3	名詞	数	*	*	*	*	*
https	名詞	一般	*	*	*	*	*
://	記号	一般	*	*	*	*	*
www	名詞	一般	*	*	*	*	*
.	記号	一般	*	*	*	*	*
fukuoka	名詞	一般	*	*	*	*	*
-	記号	一般	*	*	*	*	*
fg	名詞	一般	*	*	*	*	*
.	記号	一般	*	*	*	*	*
com	名詞	一般	*	*	*	*	*
/「	記号	一般	*	*	*	*	*
台湾	名詞	固有名詞	地域	国	*	*	台湾	タイワン	タイワン
ビジネス	名詞	一般	*	*	*	*	ビジネス	ビジネス	ビジネス
セミナー	名詞	一般	*	*	*	*	セミナー	セミナー	セミナー
in	名詞	一般	*	*	*	*	*
熊本	名詞	固有名詞	地域	一般	*	*	熊本	クマモト	クマモト
～	記号	一般	*	*	*	*	*
TSMC	名詞	一般	*	*	*	*	*
進出	名詞	サ変接続	*	*	*	*	進出	シンシュツ	シンシュツ
で	助詞	格助詞	一般	*	*	*	で	デ	デ
熊本	名詞	固有名詞	地域	一般	*	*	熊本	クマモト	クマモト
・	記号	一般	*	*	*	*	・	・	・
九州	名詞	固有名詞	地域	一般	*	*	九州	キュウシュウ	キューシュー
は	助詞	係助詞	*	*	*	*	は	ハ	ワ
どう	副詞	助詞類接続	*	*	*	*	どう	ドウ	ドー
変わる	動詞	自立	*	*	五段・ラ行	基本形	変わる	カワル	カワル
の	名詞

In [38]:
ret = _TAGGER.parse("吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。")
words = ret.split("\n")
print("\n".join(map(lambda ss: "\t".join(ss), filter(lambda ss: len(ss) > 1 and ss[1] == "名詞", map(lambda s: re.split(r"[\t,]", s), words)))))

吾輩	名詞	代名詞	一般	*	*	*	吾輩	ワガハイ	ワガハイ
猫	名詞	一般	*	*	*	*	猫	ネコ	ネコ
名前	名詞	一般	*	*	*	*	名前	ナマエ	ナマエ
どこ	名詞	代名詞	一般	*	*	*	どこ	ドコ	ドコ
見当	名詞	サ変接続	*	*	*	*	見当	ケントウ	ケントー
何	名詞	代名詞	一般	*	*	*	何	ナニ	ナニ
所	名詞	非自立	副詞可能	*	*	*	所	トコロ	トコロ
ニャーニャー	名詞	一般	*	*	*	*	*
いた事	名詞	一般	*	*	*	*	いた事	イタコト	イタコト
記憶	名詞	サ変接続	*	*	*	*	記憶	キオク	キオク
吾輩	名詞	代名詞	一般	*	*	*	吾輩	ワガハイ	ワガハイ
ここ	名詞	代名詞	一般	*	*	*	ここ	ココ	ココ
人間	名詞	一般	*	*	*	*	人間	ニンゲン	ニンゲン
もの	名詞	非自立	一般	*	*	*	もの	モノ	モノ
あと	名詞	一般	*	*	*	*	あと	アト	アト
それ	名詞	代名詞	一般	*	*	*	それ	ソレ	ソレ
書生	名詞	一般	*	*	*	*	書生	ショセイ	ショセイ
人間	名詞	一般	*	*	*	*	人間	ニンゲン	ニンゲン
中	名詞	接尾	副詞可能	*	*	*	中	チュウ	チュー
一番	名詞	副詞可能	*	*	*	*	一番	イチバン	イチバン
獰悪	名詞	形容動詞語幹	*	*	*	*	獰悪	ドウアク	ドーアク
種族	名詞	一般	*	*	*	*	種族	シュゾク	シュゾク
そう	名詞	特殊	助動詞語幹	*	*	*	そう	ソウ	ソー
書生	名詞	一般	*	*	*	*	書生	ショセイ	ショセイ
の	名詞	非自立	一般	*	*	*	の	ノ	ノ
我々	名詞	代名詞	一般	*	*	*	我々	ワレワレ	ワレワレ
話	名詞	サ変接続	*	*	*	*	話	ハナシ	ハナシ
当時	名詞	副詞可能	*	*	*	*	当時	トウジ	トージ
何	名詞	代名詞	一般	*	*	*	何	ナニ	ナニ
考	名詞	一般	*	*	*	*	考	コウ	コー
彼	名詞	代名詞	一般	*	*	*	彼	カレ	カレ
掌	名詞	一般	*	*	*	*	掌	テノヒラ	テノヒラ
スー	名詞	固有名詞	人名	名	*	*	スー	スー	スー
時	名詞	非自立	副詞可能	*	*	*	時	トキ	トキ
感じ	名詞	一般	*	*	*	*	感じ	カンジ